In [2]:
using Images
using DataFrames
using ImageMagick
using DecisionTree

In [3]:
function load_data(file, dir)
    table = readtable(file)
    n = size(table,1)
    set = zeros(Float32, n, 400)
    predict = zeros(UInt8, size(table, 1))
    
    function gray(pixelArray)
        map(pixelArray) do x
            x.r+x.g+x.b / 3.0f0
        end
    end
    
    for i in 1:n
        set[i,:] = load("$(dir)/$(table[i,1]).Bmp").data |> gray
        predict[i] = table[i,2][1] |> UInt8
    end

    (predict, set)
end

(yTrain, xTrain) = @time load_data("trainLabels.csv", "trainResized")
(yTest, xTest) = @time load_data("sampleSubmission.csv", "testResized")

;

150.647623 seconds (19.82 M allocations: 467.841 MB, 0.16% gc time)
142.976898 seconds (15.48 M allocations: 282.952 MB, 0.09% gc time)


In [ ]:
rfmodel = @time build_forest(yTrain, xTrain, 20, 50)
submission = readtable("sampleSubmission.csv")
submission[:,2] = map(Char, apply_forest(rfmodel, xTest))
writetable("submission.csv", submission)